## Maine Senate overview script

This file pulls in individual contributions to all Senate committees as well as independent expenditures to support or oppose the candidates _and_ party-coordinated expenditures to support or oppose the candidates.

The data fuels a dashboard that contains an overview of the race so far. It also provides the information to reconcile itemized contribution data with campaign totals available from the FEC.

For Susan Collins, there is potentially significant variance between these sources and the cause is not yet clear.

In [2]:
import numpy as np
import pandas as pd
import requests
import config
import os
import json
import pygsheets
from pandas.io.json import json_normalize
import time

##Schedule_a API guide: https://api.open.fec.gov/developers/#/receipts/get_schedules_schedule_a_˜

In [97]:
### INDIVIDUAL DONATIONS TO MAINE SENATE CAMPAIGNS ###

## SENATE COMMITTEE SEARCH ##
#Set search for all 2020 (two-year transaction period) Maine Senate candidates
cand_state = 'ME'
period='2020'
parameters = {'election_year':period
            ,'state':cand_state
            ,'office':'S'
            ,'api_key':config.api_key}

#Requests candidate info
r_cands = requests.get('https://api.open.fec.gov/v1/candidates/search',params=parameters).json()

#Locates and sets Committee ID from 'principal_committees' sub-array
#Output list of IDs
comm_ids=json_normalize(data=r_cands['results'],record_path='principal_committees')['committee_id'].tolist()

## FOR LOOP TO COLLECT CONTRIBUTION RECORDS ##

#Initialize dataframe collector for itemized contribs
cycle='2020'
idfs=[]
udfs=[]
commid=0

#Initialize query dictionary
itemdict = {
    'per_page':'100'
    ,'sort':'contribution_receipt_date'
    ,'api_key':config.api_key
    ,'is_individual':'true'
    ,'two_year_transaction_period':cycle
    ,'last_index':[]
    ,'last_contribution_receipt_date':[]
    ,'committee_id':comm_ids[commid]
}

#Dict for unitemized contributions
unitemdict = {
'api_key':config.api_key
,'cycle':cycle
,'per_page':'100'
,'committee_id':comm_ids[commid]
}

#Page through results for each committee id
for x in comm_ids):
    
    u_r = requests.get('https://api.open.fec.gov/v1/committee/'+comm_ids[commid]+'/totals',params=unitemdict).json()
    udf = json_normalize(u_r['results'])
    udfs.append(udf)
    
    #Get first itemized payload for a candidate
    itemr = requests.get('https://api.open.fec.gov/v1/schedules/schedule_a/',params=itemdict).json()
    
    #Print itemdict to validate
    print(itemdict)
    
    #Last page variables
    while itemr['pagination']['last_indexes'] is not None:
        
        #Store results of payload
        idf = json_normalize(itemr['results'])
        idfs.append(idf)
                
        #Assign last_index and date values, update itemdict
        last_index=pd.to_numeric(itemr['pagination']['last_indexes']['last_index'])
        last_date=itemr['pagination']['last_indexes']['last_contribution_receipt_date']
        #Update dictionary with new indices
        itemdict.update([('last_index',last_index)
                        ,('last_contribution_receipt_date',last_date)])

        #Get next payload with updated dict
        itemr = requests.get('https://api.open.fec.gov/v1/schedules/schedule_a/',params=itemdict).json()
    
    commid+=1
    
    #Update dictionary with next candidate in list and reset last indices
    itemdict.update([('committee_id',comm_ids[commid])
                     ,('last_index',[])
                     ,('last_contribution_receipt_date',[])])
    
    unitemdict.update([('committee_id',comm_ids[commid])])
    
# Concatenate all dfs
itemdf=pd.concat(idfs,sort=False,ignore_index=True)
itemdf=itemdf.drop_duplicates(subset='transaction_id')
udf=pd.concat(udfs,sort=False,ignore_index=True)
udf=udf.drop_duplicates()


#ITEMIZED DATA CLEANING#
itemdf['contributor_zip'] = itemdf['contributor_zip'].str[:5]

#Table slimming
cols = [
    'committee.name'
    ,'committee.party_full'
    ,'contribution_receipt_amount'
    ,'contribution_receipt_date'
    ,'entity_type'
    ,'contributor_city'
    ,'contributor_state'
    ,'contributor_street_1'
    ,'contributor_zip'
    ,'contributor_employer'
    ,'contributor_name'
    ,'contributor_occupation'
    ,'fec_election_type_desc'
    ,'load_date'
    ,'pdf_url'
    ,'transaction_id'
]

itemdf=itemdf[cols]
itemdf


#Create DataFrame with columns to match itemized table
unitemdf=[]
unitemdf = pd.DataFrame(columns=cols)

## Select data for unitemized df ##
unitemdf[['committee.name'
        ,'committee.party_full'
        ,'contribution_receipt_amount'
        ,'contribution_receipt_date'
        ,'fec_election_type_desc']] = udf[['committee_name'
                                        ,'party_full'
                                        ,'individual_unitemized_contributions'
                                        ,'coverage_end_date'
                                        ,'last_report_type_full']]

#Label as unitemized
unitemdf['contributor_name'] = 'Unitemized individual contributions'
unitemdf['entity_type'] = 'IND'

#Union Itemized and Unitemized contributions
ind_df = pd.concat([itemdf,unitemdf],sort=False,ignore_index=True)

#Write itemized individual results to local CSV
cwd = os.getcwd()
ind_df.to_csv(cwd+'/data/individual-senate-contributions.csv')

#Write out summary file from totals endpoint
udf.to_csv(cwd+'/senate-financial-summary.csv')

# #QA - compare itemized values to totals
# qadf = itemdf.groupby(['committee.name']).agg({'contribution_receipt_amount':['sum']})
# qadf

# totaldf = udf[['committee_name'
#              ,'individual_itemized_contributions'
#              ,'individual_unitemized_contributions']]

# qadf = qadf.merge(totaldf, left_on='committee.name',right_on='committee_name')

{'per_page': '100', 'sort': 'contribution_receipt_date', 'api_key': 'egxSs7endLz5xMuoprm5zfVZCeoyeZbO5D6HFzJz', 'is_individual': 'true', 'two_year_transaction_period': '2020', 'last_index': [], 'last_contribution_receipt_date': [], 'committee_id': 'C00710087'}
{'per_page': '100', 'sort': 'contribution_receipt_date', 'api_key': 'egxSs7endLz5xMuoprm5zfVZCeoyeZbO5D6HFzJz', 'is_individual': 'true', 'two_year_transaction_period': '2020', 'last_index': [], 'last_contribution_receipt_date': [], 'committee_id': 'C00314575'}
{'per_page': '100', 'sort': 'contribution_receipt_date', 'api_key': 'egxSs7endLz5xMuoprm5zfVZCeoyeZbO5D6HFzJz', 'is_individual': 'true', 'two_year_transaction_period': '2020', 'last_index': [], 'last_contribution_receipt_date': [], 'committee_id': 'C00709899'}
{'per_page': '100', 'sort': 'contribution_receipt_date', 'api_key': 'egxSs7endLz5xMuoprm5zfVZCeoyeZbO5D6HFzJz', 'is_individual': 'true', 'two_year_transaction_period': '2020', 'last_index': [], 'last_contribution_rece

IndexError: list index out of range

In [243]:
## SENATE CANDIDATE ID SEARCH ##
cand_ids=json_normalize(data=r_cands['results'])['candidate_id'].tolist()

### INDEPENDENT EXPENDITURE RETRIEVAL ###
#Reset committee ID
candid=0

iedict = {
    'per_page':'100'
    ,'api_key':config.api_key
    ,'two_year_transaction_period':cycle
    ,'last_index':[]
    ,'last_expenditure_date':[]
    ,'candidate_id':cand_ids[candid]
}
edfs = []

#Page through results for each committee id
for x in range(0,len(cand_ids)-1):
    
    #Get first itemized payload for a candidate
    ier = requests.get('https://api.open.fec.gov/v1/schedules/schedule_e/',params=iedict).json()
    
    #Print itemdict to validate
    print(iedict)
    
    #Last page variables
    while ier['pagination']['last_indexes'] is not None:
        
        #Store results of payload
        edf = json_normalize(ier['results'])
        edfs.append(edf)
                
        #Assign last_index and date values, update itemdict
        last_index=ier['pagination']['last_indexes']['last_index']
        last_date=ier['pagination']['last_indexes']['last_expenditure_date']
        #Update dictionary with new indices
        iedict.update([('last_index',last_index)
                        ,('last_expenditure_date',last_date)])

        #Get next payload with updated dict
        ier = requests.get('https://api.open.fec.gov/v1/schedules/schedule_e/',params=iedict).json()
    
    candid+=1
    
    #Update dictionary with next candidate in list and reset last indices
    iedict.update([('candidate_id',cand_ids[candid])
                    ,('last_index',[])
                    ,('last_expenditure_date',[])])

edf=pd.concat(edfs,sort=False,ignore_index=True)
edf=edf.drop_duplicates(subset='transaction_id')
edf

{'per_page': '100', 'api_key': 'egxSs7endLz5xMuoprm5zfVZCeoyeZbO5D6HFzJz', 'two_year_transaction_period': '2020', 'last_index': [], 'last_expenditure_date': [], 'candidate_id': 'S0ME00061'}
{'per_page': '100', 'api_key': 'egxSs7endLz5xMuoprm5zfVZCeoyeZbO5D6HFzJz', 'two_year_transaction_period': '2020', 'last_index': [], 'last_expenditure_date': [], 'candidate_id': 'S6ME00159'}
{'per_page': '100', 'api_key': 'egxSs7endLz5xMuoprm5zfVZCeoyeZbO5D6HFzJz', 'two_year_transaction_period': '2020', 'last_index': [], 'last_expenditure_date': [], 'candidate_id': 'S0ME00111'}
{'per_page': '100', 'api_key': 'egxSs7endLz5xMuoprm5zfVZCeoyeZbO5D6HFzJz', 'two_year_transaction_period': '2020', 'last_index': [], 'last_expenditure_date': [], 'candidate_id': 'S0ME00087'}
{'per_page': '100', 'api_key': 'egxSs7endLz5xMuoprm5zfVZCeoyeZbO5D6HFzJz', 'two_year_transaction_period': '2020', 'last_index': [], 'last_expenditure_date': [], 'candidate_id': 'S0ME00095'}
{'per_page': '100', 'api_key': 'egxSs7endLz5xMuopr

,action_code,action_code_full,amendment_indicator,amendment_number,back_reference_schedule_name,back_reference_transaction_id,candidate.candidate_id,candidate.idx,candidate.two_year_period,candidate_first_name,candidate_id,candidate_last_name,candidate_middle_name,candidate_name,candidate_office,candidate_office_district,candidate_office_state,candidate_party,candidate_prefix,candidate_suffix,category_code,category_code_full,committee.affiliated_committee_name,committee.candidate_ids,committee.city,committee.committee_id,committee.committee_type,committee.committee_type_full,committee.cycle,committee.cycles,committee.designation,committee.designation_full,committee.filing_frequency,committee.name,committee.organization_type,committee.organization_type_full,committee.party,committee.party_full,committee.state,committee.state_full,committee.street_1,committee.street_2,committee.treasurer_name,committee.zip,committee_id,conduit_committee_city,conduit_committee_id,conduit_committee_name,conduit_committee_state,conduit_committee_street1,conduit_committee_street2,conduit_committee_zip,dissemination_date,election_type,election_type_full,expenditure_amount,expenditure_date,expenditure_description,file_number,filer_first_name,filer_last_name,filer_middle_name,filer_prefix,filer_suffix,filing_form,image_number,independent_sign_date,independent_sign_name,is_notice,line_number,link_id,memo_code,memo_code_full,memo_text,memoed_subtotal,notary_commission_expiration_date,notary_sign_date,notary_sign_name,office_total_ytd,original_sub_id,payee_city,payee_first_name,payee_last_name,payee_middle_name,payee_name,payee_prefix,payee_state,payee_street_1,payee_street_2,payee_suffix,payee_zip,pdf_url,previous_file_number,report_type,report_year,schedule_type,schedule_type_full,sub_id,support_oppose_indicator,transaction_id
0,A,ADD,N,0,None,None,S6ME00159,99268,2020.0,SUSAN,S6ME00159,COLLINS,None,"COLLINS, SUSAN",S,00,ME,REP,None,None,None,None,NONE,[],WASHINGTON,C00573261,Q,PAC - Qualified,2020,"[2016, 2018, 2020]",U,Unauthorized,M,END CITIZENS UNITED,None,None,None,None,DC,District Of Columbia,P.O. BOX 66005,None,"COLEMAN, KIMBERLY",20035,C00573261,None,None,None,None,None,None,None,2019-10-17T00:00:00,P2020,None,5709.00,2019-10-17T00:00:00,TEXT MESSAGING ESTIMATE,1360147.0,KIMBERLY,COLEMAN,None,None,None,F24,201910189165115847,2019-10-18T00:00:00,"COLEMAN, KIMBERLY",True,24,4101820191674584207,X,None,*,True,None,None,None,10708.43,None,SEATTLE,None,None,None,TATANGO,None,WA,2211 ELLIOTT AVE,STE 200,None,981213622,http://docquery.fec.gov/cgi-bin/fecimg/?201910...,1360147.0,48,2019,SE,ITEMIZED INDEPENDENT EXPENDITURES,4101820191674589295,O,500023548
1,A,ADD,N,0,None,None,S6ME00159,99268,2020.0,SUSAN,S6ME00159,COLLINS,None,"COLLINS, SUSAN",S,00,ME,REP,None,None,004,Advertising Expenses,NONE,[],ARLINGTON,C00695619,V,PAC with Non-Contribution Account - Nonqualified,2020,[2020],U,Unauthorized,M,HEALTH JUSTICE FOR ALL,None,None,None,None,VA,Virginia,2200 WILSON BLVD,SUITE 102-554,"FEIGL-DING, ERIC DR.",22201,C00695619,None,C00695619,None,None,None,None,None,2019-10-11T00:00:00,P2020,None,3650.00,2019-10-11T00:00:00,FACEBOOK COMMUNICATION - IN-KIND VALUE (NON-CO...,1356276.0,ERIC,FEIGL-DING,None,DR.,None,F24,201910139163828324,2019-10-13T00:00:00,"FEIGL-DING, ERIC DR.",True,24,4101320191674174132,None,None,None,False,None,None,None,6291.00,None,ARLINGTON,ERIC,FEIGL-DING,None,"FEIGL-DING, ERIC DR.",DR.,VA,2200 WILSON BLVD,SUITE 102-554,None,22201,http://docquery.fec.gov/cgi-bin/fecimg/?201910...,1356276.0,48,2019,SE,ITEMIZED INDEPENDENT EXPENDITURES,4101320191674175195,O,SE.4245
2,A,ADD,N,0,None,None,S6ME00159,99268,2020.0,SUSAN,S6ME00159,COLLINS,None,"COLLINS, SUSAN",S,00,ME,REP,None,None,None,None,DNC SERVICES CORPORATION / DEMOCRATIC NATIONAL...,[],AUGUSTA,C00179408,Y,Party - Qualified,2020,"[1984, 1986, 1988, 1990, 1992, 1994, 1996, 199...",U,Unauthorized,M,MAINE DEMOCRATIC PARTY,None,None,DEM,DEMOCRATIC PARTY,ME,Maine,PO BOX 5258,320 WATER ST 3RD FLOO

array(['COLLINS', 'GIDEON'], dtype=object)

In [40]:
# #Google Credentials
# gc = pygsheets.authorize(service_file=cwd+'/../me-congress-2020-creds.json')

# #Select sheet and worksheet
# sh = gc.open('maine-senate-2020')
# # sh = gc.open_by_key('1AKrgHT9NLpoddV16B7_M_0PEjJmMQAGtXJUnLCTDHjA')
# wks = sh[0]

# #Clear sheet before load
# wks.clear(start='A1',fields='*')

# #Write contribs dataframe to sheet
# wks.set_dataframe(df_cull,(1,1))

HttpError: <HttpError 400 when requesting https://sheets.googleapis.com/v4/spreadsheets/12bHf1qEtKtGGje0a3lBJJ8r-IvYcs3yl5jxbP6jNMCo/values/Individual%20itemized%21A3334%3AP6667?valueInputOption=USER_ENTERED&alt=json returned "Range ('Individual itemized'!A3334:P6667) exceeds grid limits. Max rows: 3333, max columns: 120">